In [1]:
import spacy
import pandas as pd
import os

In [2]:
from lxml import etree
from datetime import datetime

In [3]:
def month_dates(start, end):
    f = lambda date: date.month + 12 * date.year

    res = []
    for tot_m in range(f(start)-1, f(end)):
        y, m = divmod(tot_m, 12)
        res.append(str(y) + '/' + '%02d' % (m+1))
    
    return res

In [4]:
def get_date(article):
    """
    This method returns the date of the article
    """
    str_date = article.find('entity').find('meta').find('issue_date').text
    return datetime.strptime(str_date, '%d/%m/%Y')

In [5]:
def get_articles_in_file(file, start_date, end_date):
    articles = []  
    for article in file.iter('article'):
        if article.find('entity') is not None:
            a = ''
            date = get_date(article)
            if start_date <= date <= end_date:
                for entity in article.iter('entity'):
                    a += entity.findtext('full_text') + ' '
                articles.append(date.strftime('%d/%m/%Y') + ' ' + a)
    return articles

In [6]:
def get_articles(path, start_date, end_date):
    articles = []
    for m_date in month_dates(start_date, end_date):
        try:
            file = etree.parse(path + m_date + '.xml')
            articles.append(get_articles_in_file(file, start_date, end_date))
        except (FileNotFoundError, IOError):
            pass
    return [a for file in articles for a in file]  

In [7]:
path = '/home/mbanga/Desktop/JDG/'
start_date =  datetime(1990, 1, 1)
end_date = datetime(1990, 1, 31)

In [8]:
articles = get_articles(path, start_date, end_date)

In [9]:
len(articles)

3434

In [10]:
import fr_core_news_sm
import enchant

In [11]:
nlp = fr_core_news_sm.load()

In [12]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuaiton or whitespace
    """
    
    return token.is_punct or token.is_space

In [13]:
def is_french(word):
    """
    helper function to eliminate tokens that
    are not french words.
    """
    d = enchant.Dict('fr_FR')
    return d.check(word)

In [14]:
def lemmatized_corpus(corpus):
    """
    generator function to use spaCy to parse articles,
    lemmatize the text, and yield sentences
    """
    j = 0
    i = 0
    for parsed_article in nlp.pipe(corpus, 
                                   batch_size=50, n_threads=7):
        
        date = parsed_article[0].text + ' '
        yield date + u' '.join([token.lemma_ for token in parsed_article
                             if not punct_space(token) and is_french(token.text) and not token.is_stop])

In [15]:
# Takes too long
lemmatized_articles = []
for lemmatized in lemmatized_corpus(articles[:300]):
    lemmatized_articles.append(lemmatized)

In [16]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary, MmCorpus

import pyLDAvis
import pyLDAvis.gensim
import warnings
#import cPickle as pickle

In [17]:
corpus = lemmatized_articles
# learn the dictionnary by iterating over all of the articles
dico = Dictionary([article.split() for article in corpus])

# filter tokens that are very rare or too common from
# the dictionary 
dico.filter_extremes(no_below=10, no_above=0.4)

# reassign integer lda
dico.compactify()

In [18]:
def bow_generator(corpus):
    """
    generator function to read articles from a file
    and yield a bag-of-words representation
    """
    for article in corpus:
        yield dico.doc2bow(article.split())

In [19]:
# generate bag-of-word representations for
# all reviews and save them as a matrix
project_path = '/home/mbanga/Epfl/AppliedDataAnalysis/ADA2017_GroupWork/Project/'
MmCorpus.serialize(os.path.join(project_path, 'corpus.mm'),
                                bow_generator(corpus))

bow_corpus = MmCorpus(os.path.join(project_path, 'corpus.mm'))

In [20]:
lda_model_filepath = os.path.join(project_path, 'lda_model_all')

In [82]:
if 0 == 1:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(bow_corpus,
                           num_topics=40,
                           id2word=dico,
                           workers=4)
        
        lda.save(lda_model_filepath)

#load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

In [83]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
    
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
    
    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [90]:
explore_topic(topic_number=1)

term                 frequency

pays                 0.017
dollar               0.017
mardi                0.016
économique           0.014
ministre             0.012
aider                0.011
janvier              0.011
marcher              0.011
vouloir              0.011
prix                 0.010
américain            0.010
2                    0.010
mois                 0.010
plan                 0.009
décembre             0.009
officiel             0.009
présider             0.009
placer               0.009
vol                  0.009
sur                  0.008
se                   0.008
lundi                0.008
annoncer             0.007
4                    0.007
en                   0.007


In [25]:
len(corpus)

300